# Performance Optimization with SUBMARIT

This notebook demonstrates techniques for optimizing SUBMARIT performance with large datasets.

## Table of Contents
1. [Memory-Efficient Techniques](#memory)
2. [Computational Optimization](#computation)
3. [Parallel and Distributed Processing](#parallel)
4. [Approximate Methods](#approximate)
5. [Benchmarking and Profiling](#benchmarking)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import psutil
import os
from memory_profiler import memory_usage
from scipy.sparse import csr_matrix, coo_matrix
from scipy.spatial.distance import pdist, squareform

# SUBMARIT imports
from submarit.core import (
    create_substitution_matrix,
    create_sparse_substitution_matrix
)
from submarit.algorithms import LocalSearch, MiniBatchLocalSearch
from submarit.utils import Timer

# Set random seed
np.random.seed(42)

print("Performance optimization notebook ready!")
print(f"Available memory: {psutil.virtual_memory().available / 1e9:.1f} GB")
print(f"CPU cores: {psutil.cpu_count()}")

## 1. Memory-Efficient Techniques <a id='memory'></a>

Strategies for reducing memory usage with large substitution matrices.

In [ ]:
# Generate datasets of increasing size
sizes = [100, 500, 1000, 2000, 5000]
n_features = 20

memory_results = []

for n in sizes:
    print(f"\nProcessing {n} products...")
    X = np.random.randn(n, n_features).astype(np.float32)  # Use float32 to save memory
    
    # Dense matrix memory
    dense_memory = (n * n * 4) / 1e6  # 4 bytes per float32
    
    # Sparse matrix estimation (keep top 10%)
    sparsity = 0.9
    sparse_memory = (n * n * (1 - sparsity) * 12) / 1e6  # ~12 bytes per entry
    
    memory_results.append({
        'n_products': n,
        'dense_mb': dense_memory,
        'sparse_mb': sparse_memory,
        'ratio': dense_memory / sparse_memory
    })
    
    print(f"Dense: {dense_memory:.1f} MB, Sparse: {sparse_memory:.1f} MB, Ratio: {dense_memory/sparse_memory:.1f}x")

# Visualize memory usage
df_memory = pd.DataFrame(memory_results)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(df_memory['n_products'], df_memory['dense_mb'], 'o-', label='Dense', linewidth=2)
ax1.plot(df_memory['n_products'], df_memory['sparse_mb'], 's-', label='Sparse (10%)', linewidth=2)
ax1.set_xlabel('Number of Products')
ax1.set_ylabel('Memory (MB)')
ax1.set_title('Memory Usage Comparison')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

ax2.bar(df_memory['n_products'].astype(str), df_memory['ratio'])
ax2.set_xlabel('Number of Products')
ax2.set_ylabel('Memory Savings Ratio')
ax2.set_title('Sparse vs Dense Memory Savings')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Demonstration: Sparse matrix creation and usage
n_demo = 1000
X_demo = np.random.randn(n_demo, n_features)

print("Creating substitution matrices...")

# Dense matrix
start = time.time()
S_dense = create_substitution_matrix(X_demo, metric='euclidean')
time_dense = time.time() - start
mem_dense = S_dense.nbytes / 1e6

# Sparse matrix (keep top 10%)
start = time.time()
S_sparse = create_sparse_substitution_matrix(X_demo, threshold=0.1, metric='euclidean')
time_sparse = time.time() - start
mem_sparse = (S_sparse.data.nbytes + S_sparse.indices.nbytes + S_sparse.indptr.nbytes) / 1e6

print(f"\nDense matrix:")
print(f"  Creation time: {time_dense:.2f}s")
print(f"  Memory usage: {mem_dense:.1f} MB")

print(f"\nSparse matrix:")
print(f"  Creation time: {time_sparse:.2f}s")
print(f"  Memory usage: {mem_sparse:.1f} MB")
print(f"  Sparsity: {1 - S_sparse.nnz / (n_demo**2):.1%}")
print(f"  Memory reduction: {(1 - mem_sparse/mem_dense)*100:.1f}%")

In [ ]:
# Memory-mapped arrays for very large datasets
def create_mmap_substitution_matrix(X, filename='submatrix.dat', dtype=np.float32):
    """Create memory-mapped substitution matrix."""
    n = len(X)
    
    # Create memory-mapped array
    S = np.memmap(filename, dtype=dtype, mode='w+', shape=(n, n))
    
    # Compute in chunks to avoid memory overflow
    chunk_size = min(100, n)
    
    for i in range(0, n, chunk_size):
        end_i = min(i + chunk_size, n)
        for j in range(0, n, chunk_size):
            end_j = min(j + chunk_size, n)
            
            # Compute chunk
            dists = pdist(np.vstack([X[i:end_i], X[j:end_j]]), metric='euclidean')
            dist_matrix = squareform(dists)
            
            # Extract relevant part
            S[i:end_i, j:end_j] = dist_matrix[:end_i-i, end_i-i:end_i-i+end_j-j]
    
    # Ensure symmetry
    S[:] = np.maximum(S, S.T)
    
    return S

# Demo with smaller dataset
n_mmap = 500
X_mmap = np.random.randn(n_mmap, n_features)

print("Creating memory-mapped substitution matrix...")
S_mmap = create_mmap_substitution_matrix(X_mmap, 'demo_submatrix.dat')

print(f"Memory-mapped matrix created")
print(f"File size: {os.path.getsize('demo_submatrix.dat') / 1e6:.1f} MB")
print(f"Can handle much larger datasets without loading into RAM")

# Clean up
del S_mmap
os.remove('demo_submatrix.dat')

## 2. Computational Optimization <a id='computation'></a>

Speed up computations using various techniques.

In [ ]:
# Vectorization comparison
n_test = 1000
X_test = np.random.randn(n_test, n_features)

# Naive implementation with loops
def naive_substitution_matrix(X):
    n = len(X)
    S = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            dist = np.sqrt(np.sum((X[i] - X[j])**2))
            S[i, j] = S[j, i] = dist
    return S

# Vectorized implementation
def vectorized_substitution_matrix(X):
    return squareform(pdist(X, metric='euclidean'))

# Time comparison (with smaller dataset for naive)
n_small = 200
X_small = X_test[:n_small]

print("Timing comparison (200 products):")
start = time.time()
S_naive = naive_substitution_matrix(X_small)
time_naive = time.time() - start
print(f"Naive (loops): {time_naive:.3f}s")

start = time.time()
S_vectorized = vectorized_substitution_matrix(X_small)
time_vectorized = time.time() - start
print(f"Vectorized: {time_vectorized:.3f}s")
print(f"Speedup: {time_naive/time_vectorized:.1f}x")

# Verify results are the same
print(f"\nResults match: {np.allclose(S_naive, S_vectorized)}")

In [ ]:
# Using Numba for JIT compilation
try:
    from numba import jit, prange
    
    @jit(nopython=True, parallel=True)
    def numba_euclidean_distances(X):
        n = X.shape[0]
        distances = np.zeros((n, n))
        
        for i in prange(n):
            for j in range(i+1, n):
                dist = 0.0
                for k in range(X.shape[1]):
                    dist += (X[i, k] - X[j, k])**2
                dist = np.sqrt(dist)
                distances[i, j] = distances[j, i] = dist
        
        return distances
    
    # Warm-up JIT
    _ = numba_euclidean_distances(X_small[:10])
    
    # Time comparison
    print("\nNumba JIT compilation:")
    start = time.time()
    S_numba = numba_euclidean_distances(X_small)
    time_numba = time.time() - start
    print(f"Numba (parallel): {time_numba:.3f}s")
    print(f"Speedup vs naive: {time_naive/time_numba:.1f}x")
    print(f"Results match: {np.allclose(S_numba, S_vectorized)}")
    
except ImportError:
    print("Numba not installed. Install with: pip install numba")

In [ ]:
# Optimized Local Search implementation
class OptimizedLocalSearch:
    """Local Search with performance optimizations."""
    
    def __init__(self, n_clusters, max_iter=100, tol=1e-4):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        
    def fit_predict(self, S):
        n = len(S)
        
        # Initialize clusters
        clusters = np.random.randint(0, self.n_clusters, n)
        
        # Pre-allocate arrays
        cluster_costs = np.zeros(self.n_clusters)
        best_improvement = np.inf
        
        for iteration in range(self.max_iter):
            changed = False
            total_improvement = 0
            
            # Randomize order for better convergence
            order = np.random.permutation(n)
            
            for i in order:
                old_cluster = clusters[i]
                
                # Vectorized cost computation
                for k in range(self.n_clusters):
                    mask = clusters == k
                    if k != old_cluster:
                        mask[i] = True
                    else:
                        mask[i] = False
                    
                    cluster_costs[k] = S[i, mask].sum() if mask.any() else 0
                
                # Find best cluster
                best_cluster = np.argmin(cluster_costs)
                
                if best_cluster != old_cluster:
                    improvement = cluster_costs[old_cluster] - cluster_costs[best_cluster]
                    if improvement > 0:
                        clusters[i] = best_cluster
                        changed = True
                        total_improvement += improvement
            
            if not changed or total_improvement < self.tol:
                break
        
        self.n_iter_ = iteration + 1
        return clusters

# Compare with standard implementation
S_test = S_dense[:500, :500]

print("Algorithm comparison (500 products, 5 clusters):")

# Standard
start = time.time()
ls_standard = LocalSearch(n_clusters=5, max_iter=50, n_restarts=1)
clusters_standard = ls_standard.fit_predict(S_test)
time_standard = time.time() - start

# Optimized
start = time.time()
ls_optimized = OptimizedLocalSearch(n_clusters=5, max_iter=50)
clusters_optimized = ls_optimized.fit_predict(S_test)
time_optimized = time.time() - start

print(f"Standard: {time_standard:.3f}s")
print(f"Optimized: {time_optimized:.3f}s")
print(f"Speedup: {time_standard/time_optimized:.1f}x")

## 3. Parallel and Distributed Processing <a id='parallel'></a>

Leverage multiple cores and distributed computing.

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

# Parallel distance computation
def compute_distance_chunk(X, i_start, i_end, j_start, j_end):
    """Compute a chunk of the distance matrix."""
    chunk = np.zeros((i_end - i_start, j_end - j_start))
    
    for i in range(i_end - i_start):
        for j in range(j_end - j_start):
            if i_start + i != j_start + j:  # Skip diagonal
                chunk[i, j] = np.linalg.norm(X[i_start + i] - X[j_start + j])
    
    return i_start, i_end, j_start, j_end, chunk

def parallel_distance_matrix(X, n_jobs=-1, chunk_size=100):
    """Compute distance matrix in parallel."""
    n = len(X)
    S = np.zeros((n, n))
    
    # Create chunks
    chunks = []
    for i in range(0, n, chunk_size):
        for j in range(i, n, chunk_size):  # Upper triangle only
            chunks.append((X, i, min(i + chunk_size, n), 
                          j, min(j + chunk_size, n)))
    
    # Process in parallel
    results = Parallel(n_jobs=n_jobs)(
        delayed(compute_distance_chunk)(*chunk) for chunk in chunks
    )
    
    # Assemble results
    for i_start, i_end, j_start, j_end, chunk in results:
        S[i_start:i_end, j_start:j_end] = chunk
        if i_start != j_start:  # Mirror for lower triangle
            S[j_start:j_end, i_start:i_end] = chunk.T
    
    return S

# Test parallel computation
n_parallel = 500
X_parallel = np.random.randn(n_parallel, n_features)

print(f"Computing distance matrix for {n_parallel} products...")
print(f"Available cores: {multiprocessing.cpu_count()}")

# Serial
start = time.time()
S_serial = squareform(pdist(X_parallel))
time_serial = time.time() - start

# Parallel
start = time.time()
S_parallel = parallel_distance_matrix(X_parallel, n_jobs=-1, chunk_size=100)
time_parallel = time.time() - start

print(f"\nSerial: {time_serial:.3f}s")
print(f"Parallel: {time_parallel:.3f}s")
print(f"Speedup: {time_serial/time_parallel:.1f}x")
print(f"Results match: {np.allclose(S_serial, S_parallel)}")

In [ ]:
# Parallel clustering with different random initializations
def run_single_clustering(S, n_clusters, seed):
    """Run single clustering with specific seed."""
    ls = LocalSearch(n_clusters=n_clusters, n_restarts=1, random_state=seed)
    clusters = ls.fit_predict(S)
    return clusters, ls.objective_

# Test data
S_test = S_dense[:300, :300]
n_runs = 20

print(f"Running {n_runs} clusterings in parallel...")

# Parallel execution
start = time.time()
parallel_results = Parallel(n_jobs=-1)(
    delayed(run_single_clustering)(S_test, 5, seed) 
    for seed in range(n_runs)
)
time_parallel = time.time() - start

# Find best result
best_idx = np.argmin([obj for _, obj in parallel_results])
best_clusters, best_objective = parallel_results[best_idx]

print(f"\nCompleted in {time_parallel:.2f}s")
print(f"Best objective: {best_objective:.3f}")
print(f"Average time per run: {time_parallel/n_runs:.3f}s")

## 4. Approximate Methods <a id='approximate'></a>

Trade accuracy for speed with approximate algorithms.

In [ ]:
# Approximate nearest neighbors using LSH
from sklearn.random_projection import GaussianRandomProjection

class ApproximateSubstitutionMatrix:
    """Create approximate substitution matrix using random projections."""
    
    def __init__(self, n_components=50):
        self.n_components = n_components
        self.rp = GaussianRandomProjection(n_components=n_components)
    
    def fit_transform(self, X):
        # Project to lower dimension
        X_projected = self.rp.fit_transform(X)
        
        # Compute distances in lower dimension
        S_approx = squareform(pdist(X_projected))
        
        return S_approx

# Test approximation quality
n_test = 1000
X_test = np.random.randn(n_test, 50)  # Higher dimensional data

print("Computing exact substitution matrix...")
start = time.time()
S_exact = create_substitution_matrix(X_test)
time_exact = time.time() - start

print("\nTesting different approximation levels:")
components = [10, 20, 30, 40]
results = []

for n_comp in components:
    asm = ApproximateSubstitutionMatrix(n_components=n_comp)
    
    start = time.time()
    S_approx = asm.fit_transform(X_test)
    time_approx = time.time() - start
    
    # Calculate approximation error
    error = np.mean(np.abs(S_exact - S_approx)) / np.mean(S_exact)
    
    results.append({
        'components': n_comp,
        'time': time_approx,
        'speedup': time_exact / time_approx,
        'error': error * 100
    })
    
    print(f"  {n_comp} components: {time_approx:.3f}s (speedup: {time_exact/time_approx:.1f}x, error: {error*100:.1f}%)")

# Visualize trade-off
df_results = pd.DataFrame(results)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(df_results['components'], df_results['speedup'], 'o-', markersize=8)
ax1.set_xlabel('Number of Components')
ax1.set_ylabel('Speedup Factor')
ax1.set_title('Speedup vs Approximation Level')
ax1.grid(True, alpha=0.3)

ax2.plot(df_results['components'], df_results['error'], 's-', color='red', markersize=8)
ax2.set_xlabel('Number of Components')
ax2.set_ylabel('Approximation Error (%)')
ax2.set_title('Error vs Approximation Level')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Mini-batch Local Search for large datasets
class FastMiniBatchLocalSearch:
    """Optimized mini-batch implementation."""
    
    def __init__(self, n_clusters, batch_size=100, n_init=3, subsample_ratio=0.1):
        self.n_clusters = n_clusters
        self.batch_size = batch_size
        self.n_init = n_init
        self.subsample_ratio = subsample_ratio
    
    def fit_predict(self, S):
        n = len(S)
        
        # Phase 1: Quick clustering on subsample
        subsample_size = int(n * self.subsample_ratio)
        subsample_idx = np.random.choice(n, subsample_size, replace=False)
        S_sub = S[subsample_idx][:, subsample_idx]
        
        # Run standard algorithm on subsample
        ls = LocalSearch(n_clusters=self.n_clusters, n_restarts=self.n_init)
        clusters_sub = ls.fit_predict(S_sub)
        
        # Phase 2: Assign remaining points
        clusters = np.zeros(n, dtype=int)
        clusters[subsample_idx] = clusters_sub
        
        # Find cluster centers
        centers = []
        for k in range(self.n_clusters):
            mask = subsample_idx[clusters_sub == k]
            if len(mask) > 0:
                center_idx = mask[np.argmin(S[mask][:, mask].sum(axis=1))]
                centers.append(center_idx)
            else:
                centers.append(np.random.randint(n))
        
        # Assign remaining points to nearest center
        unassigned = np.setdiff1d(np.arange(n), subsample_idx)
        for i in unassigned:
            distances = [S[i, c] for c in centers]
            clusters[i] = np.argmin(distances)
        
        return clusters

# Compare performance
n_large = 2000
X_large = np.random.randn(n_large, 30)
S_large = create_substitution_matrix(X_large)

print(f"Clustering {n_large} products...")

# Standard approach
start = time.time()
ls_standard = LocalSearch(n_clusters=10, n_restarts=5)
clusters_standard = ls_standard.fit_predict(S_large)
time_standard = time.time() - start

# Fast mini-batch
start = time.time()
mbls_fast = FastMiniBatchLocalSearch(n_clusters=10, subsample_ratio=0.2)
clusters_fast = mbls_fast.fit_predict(S_large)
time_fast = time.time() - start

# Evaluate quality
from sklearn.metrics import adjusted_rand_score
agreement = adjusted_rand_score(clusters_standard, clusters_fast)

print(f"\nStandard: {time_standard:.2f}s")
print(f"Fast mini-batch: {time_fast:.2f}s")
print(f"Speedup: {time_standard/time_fast:.1f}x")
print(f"Agreement (ARI): {agreement:.3f}")

## 5. Benchmarking and Profiling <a id='benchmarking'></a>

Tools for measuring and optimizing performance.

In [ ]:
# Comprehensive benchmarking suite
def benchmark_algorithm(algorithm, S, name, n_runs=5):
    """Benchmark an algorithm."""
    times = []
    
    for _ in range(n_runs):
        start = time.time()
        _ = algorithm.fit_predict(S)
        times.append(time.time() - start)
    
    return {
        'name': name,
        'mean_time': np.mean(times),
        'std_time': np.std(times),
        'min_time': np.min(times),
        'max_time': np.max(times)
    }

# Test different algorithms
n_bench = 500
X_bench = np.random.randn(n_bench, 20)
S_bench = create_substitution_matrix(X_bench)

algorithms = [
    (LocalSearch(n_clusters=5, n_restarts=10), "Standard Local Search"),
    (LocalSearch(n_clusters=5, n_restarts=5), "Local Search (5 restarts)"),
    (MiniBatchLocalSearch(n_clusters=5, batch_size=100), "Mini-Batch Local Search"),
    (FastMiniBatchLocalSearch(n_clusters=5, subsample_ratio=0.3), "Fast Mini-Batch"),
]

print("Benchmarking algorithms...")
results = []

for algo, name in algorithms:
    result = benchmark_algorithm(algo, S_bench, name, n_runs=3)
    results.append(result)
    print(f"{name}: {result['mean_time']:.3f}s ± {result['std_time']:.3f}s")

# Visualize results
df_bench = pd.DataFrame(results)

plt.figure(figsize=(10, 6))
x = np.arange(len(df_bench))
plt.bar(x, df_bench['mean_time'], yerr=df_bench['std_time'], capsize=5)
plt.xticks(x, df_bench['name'], rotation=45, ha='right')
plt.ylabel('Time (seconds)')
plt.title('Algorithm Performance Comparison')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Memory profiling
def memory_profile_function(func, *args, **kwargs):
    """Profile memory usage of a function."""
    mem_usage = memory_usage((func, args, kwargs))
    return max(mem_usage) - min(mem_usage)

# Test memory usage
sizes = [100, 200, 500, 1000]
memory_results = []

print("Memory usage analysis:")
for n in sizes:
    X = np.random.randn(n, 20).astype(np.float32)
    
    # Dense matrix memory
    mem_dense = memory_profile_function(create_substitution_matrix, X)
    
    # Sparse matrix memory
    mem_sparse = memory_profile_function(create_sparse_substitution_matrix, X, threshold=0.1)
    
    memory_results.append({
        'n': n,
        'dense_mb': mem_dense,
        'sparse_mb': mem_sparse,
        'ratio': mem_dense / mem_sparse if mem_sparse > 0 else np.inf
    })
    
    print(f"  n={n}: Dense={mem_dense:.1f}MB, Sparse={mem_sparse:.1f}MB, Ratio={memory_results[-1]['ratio']:.1f}x")

# Plot memory scaling
df_mem = pd.DataFrame(memory_results)

plt.figure(figsize=(10, 6))
plt.plot(df_mem['n'], df_mem['dense_mb'], 'o-', label='Dense', markersize=8)
plt.plot(df_mem['n'], df_mem['sparse_mb'], 's-', label='Sparse', markersize=8)
plt.xlabel('Number of Products')
plt.ylabel('Memory Usage (MB)')
plt.title('Memory Usage Scaling')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Performance recommendations based on dataset size
def recommend_approach(n_products, n_features, available_memory_gb):
    """Recommend optimization approach based on dataset characteristics."""
    
    # Estimate memory requirements (MB)
    dense_memory = (n_products ** 2 * 4) / 1e6  # float32
    sparse_memory = dense_memory * 0.1  # Assume 90% sparsity
    
    recommendations = []
    
    # Memory recommendations
    if dense_memory < available_memory_gb * 1000 * 0.5:  # Use 50% of available
        recommendations.append("✓ Dense matrix feasible")
    elif sparse_memory < available_memory_gb * 1000 * 0.5:
        recommendations.append("⚠ Use sparse matrix representation")
    else:
        recommendations.append("❌ Use memory-mapped arrays or distributed computing")
    
    # Algorithm recommendations
    if n_products < 1000:
        recommendations.append("✓ Standard Local Search with multiple restarts")
    elif n_products < 10000:
        recommendations.append("⚠ Use Mini-Batch Local Search or parallel processing")
    else:
        recommendations.append("❌ Use approximate methods or subsampling")
    
    # Optimization recommendations
    if n_features > 100:
        recommendations.append("💡 Consider dimensionality reduction")
    
    if n_products > 5000:
        recommendations.append("💡 Use parallel processing (-1 jobs)")
    
    return recommendations

# Test recommendations
test_cases = [
    (500, 20, 8),
    (5000, 50, 16),
    (20000, 100, 32),
    (100000, 200, 64)
]

print("Performance Recommendations:")
print("=" * 60)

for n_prod, n_feat, mem_gb in test_cases:
    print(f"\nDataset: {n_prod:,} products × {n_feat} features, {mem_gb}GB RAM")
    recommendations = recommend_approach(n_prod, n_feat, mem_gb)
    for rec in recommendations:
        print(f"  {rec}")

## Summary

### Key Performance Optimization Strategies:

1. **Memory Optimization**:
   - Use sparse matrices when >80% zeros
   - Memory-mapped arrays for very large datasets
   - Float32 instead of float64 when precision allows

2. **Computational Optimization**:
   - Vectorize operations using NumPy
   - JIT compilation with Numba for custom functions
   - Avoid redundant computations

3. **Parallel Processing**:
   - Use all available cores with `n_jobs=-1`
   - Parallel distance matrix computation
   - Multiple random initializations in parallel

4. **Approximate Methods**:
   - Random projections for dimensionality reduction
   - Subsampling for initial clustering
   - Trade accuracy for speed when appropriate

5. **Profiling and Monitoring**:
   - Always profile before optimizing
   - Monitor memory usage
   - Benchmark different approaches

### Decision Tree:

- **< 1,000 products**: Use standard dense matrix
- **1,000 - 10,000 products**: Consider sparse matrices and parallel processing
- **10,000 - 100,000 products**: Use mini-batch algorithms and approximations
- **> 100,000 products**: Distributed computing or specialized big data tools